In [4]:
import gym
env = gym.make('MountainCar-v0')
print('观测空间= {}'. format(env.observation_space))
print('动作空间= {}'. format(env.action_space))
print('观测范围= {} ~ {}'. format(env.observation_space.low,env.observation_space.high))
print('动作数= {}'. format(env.action_space.n))

观测空间= Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
动作空间= Discrete(3)
观测范围= [-1.2  -0.07] ~ [0.6  0.07]
动作数= 3


In [5]:
class BespokeAgent:
    def __init__(self, env):
        pass
    def decide(self, observation): # 决策
        position, velocity = observation
        lb = min(-0.09 * (position + 0.25) ** 2 + 0.03,
        0.3 * (position + 0.9) ** 4 - 0.008)
        ub = -0.07 * (position + 0.38) ** 2 + 0.07
        if lb < velocity < ub:
            action = 2
        else:
            action = 0
        return action # 返回动作
    def learn(self, *args): # 学习
        pass
agent = BespokeAgent(env)

In [6]:
def play_montecarlo(env, agent, render=False, train=False):
    episode_reward = 0. # 记录回合总奖励，初始化为0
    observation = env.reset() # 重置游戏环境，开始新回合
    while True: # 不断循环，直到回合结束
        if render: # 判断是否显示
            env.render() # 显示图形界面，图形界面可以用env.close() 语句关闭
        action = agent.decide(observation)
        next_observation, reward, done, _ = env.step(action) # 执行动作
        episode_reward += reward # 收集回合奖励
        if train: # 判断是否训练智能体
            agent.learn(observation, action, reward, done) # 学习
        if done: # 回合结束，跳出循环
            break
        observation = next_observation
    return episode_reward # 返回回合总奖励

In [7]:
env.seed(0) # 设置随机数种子,只是为了让结果可以精确复现,一般情况下可删去
episode_reward = play_montecarlo(env, agent, render=True)
print('回合奖励= {}'. format(episode_reward))
env.close() # 此语句可关闭图形界面

/opt/anaconda3/lib/python3.9/site-packages/gym/core.py:172: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed) instead.
  deprecation(


回合奖励= -123.0


In [9]:
import numpy as np
episode_rewards = [play_montecarlo(env, agent) for _ in range(100)]
print('平均回合奖励= {}'. format(np.mean(episode_rewards)))

平均回合奖励= -106.24


1. 基于策略的（policy-based）：智能体会制定一套动作策略，即确定在给定状态下需要采取何种动作,并根据这个策略进行操作。强化学习算法直接对策略进行优化使制定的策略能够获得最大的奖励。
2. 基于价值的（valued-based）：智能体不需要制定显式的策略，它维护一个价值表格或者价值函数，并通过这个价值表格或价值函数来执行使得价值最大化的动作。
3. 有模型（model-based）结构：智能体通过学习状态的转移来进行决策。(对环境建模)
4. 免模型（model-free）结构：智能体没有直接估计状态的转移，也没有得到环境的具体转移变量，它通过学习价值函数或者策略网络进行决策。